In [ ]:
!pip install openai
!pip install sounddevice
!pip install python-dotenv
!pip install scipy
!pip install requests

In [ ]:
from os import environ
from dotenv.main import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]
IMGFLIP_PASSWORD = environ["IMGFLIP_PASSWORD"]

# Uncomment for troubleshooting
# print(OPENAI_API_KEY)
# print(IMGFLIP_PASSWORD)

In [ ]:
import sounddevice
from scipy.io.wavfile import write
sampleRate = 44100
print ("Recording started...")
audio = sounddevice.rec(int(15 * sampleRate), sampleRate, channels=1)
sounddevice.wait()
print ("Recording ended.")
write("recording.wav", sampleRate, audio)


In [34]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

# Re-use this file for development, troubleshooting, demonstration, etc
# file_path = "./proven_recording.wav"
file_path = "./recording.wav"

with open(file_path, "rb") as audio_file:
    response = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file,
        language="en",
        response_format="text"
    )

if response:
    print("Transcription:", response)

transcription = response

Transcription: Create a meme using the one does not simply image template. The top text should say an engineer does not simply. The bottom text should say create a meme.



In [35]:
from openai import OpenAI

def chat(prompt):
    client = OpenAI()
    response = client.chat.completions.with_raw_response.create(
        messages=[{
            "role": "user",
            "content": prompt
        }],
        model="gpt-3.5-turbo",
    )
    completion = response.parse()
    result = completion.choices[0].message.content
    return result
    

In [36]:
memeName = chat("What is the name of the meme that the user is asking about in the phrase that follows the # character? Emit just the meme name, nothing else. # " + transcription)
print("Meme name: " + memeName)

memeNumber = chat("The web page at  https://imgflip.com/popular-meme-ids contains a table of meme image IDs and descriptions. What is the ID of the '" + memeName + "' meme? Emit just the ID number, nothing else.")
print("Meme number: " + memeNumber)

topText = chat("In the phrase that follows the # character, What 'top text' does the user want to be on the top of meme? Emit just the top text, nothing else. # " + transcription)
print("Top text: " + topText)

bottomText = chat("In the phrase that follows the # character, What text does the user want to put on the bottom of meme? Emit just the text, nothing else. # " + transcription)
print("Bottom text: " + bottomText)

Meme name: One does not simply
Meme number: 61579
Top text: an engineer does not simply
Bottom text: create a meme


In [37]:
import requests
import urllib

url = "https://api.imgflip.com/caption_image"

params = {
    'username':'natewaddoups',
    'password': IMGFLIP_PASSWORD,
    'template_id': memeNumber,
    'text0':topText,
    'text1':bottomText
}
response = requests.request('POST',url,params=params).json()
print(response)

import webbrowser
webbrowser.open(response["data"]["url"])


{'success': True, 'data': {'url': 'https://i.imgflip.com/981y6c.jpg', 'page_url': 'https://imgflip.com/i/981y6c'}}


True